In [ ]:
import cv2
import numpy as np
import imutils
import tkinter as tk
from threading import Thread

number_color_ranges = {
    'PHP 50': ((0, 100, 150), (20, 255, 255)),    
    'PHP 20': ((100, 20, 150), (180, 100, 255)),  
    'PHP 100': ((120, 50, 50), (255, 120, 120)),  
    'PHP 200': ((0, 150, 0), (90, 255, 90)),      
    'PHP 500': ((0, 170, 200), (60, 255, 255)),   
    'PHP 1000': ((200, 170, 140), (255, 255, 255))
}

def boxes_overlap(box1, box2):
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2
    return not ((x1 > x2 + w2) or (x2 > x1 + w1) or (y1 > y2 + h2) or (y2 > y1 + h1))

def detect_bills_by_number_color(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    edged = cv2.Canny(gray, 50, 150)
    contours, _ = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    detected_bills = []
    total_value = 0
    detected_boxes = []

    for contour in contours:
        perimeter = cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, 0.04 * perimeter, True)
        x, y, w, h = cv2.boundingRect(approx)
        area = cv2.contourArea(approx)
        aspect_ratio = w / float(h)

        if area > 15000 and 1.5 < aspect_ratio < 2.5:
            overlap_found = any(boxes_overlap((x, y, w, h), db) for db in detected_boxes)
            if overlap_found:
                continue

            roi = frame[y:y+int(h*0.3), x:x+int(w*0.3)]
            if roi.size == 0:
                continue

            hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
            best_match = None
            max_pixels = 0

            for bill, (lower, upper) in number_color_ranges.items():
                lower_np = np.array(lower)
                upper_np = np.array(upper)
                mask = cv2.inRange(hsv_roi, lower_np, upper_np)
                pixel_count = cv2.countNonZero(mask)

                if pixel_count > max_pixels:
                    max_pixels = pixel_count
                    best_match = bill

            if max_pixels > 500:
                value = int(best_match.split()[1])
                detected_bills.append(best_match)
                detected_boxes.append((x, y, w, h))
                total_value += value

                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.putText(frame, best_match, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    return frame, detected_bills, total_value

def update_gui():
    total_bills_var.set(f"Total Bills: {len(detected_bills)}")
    total_value_var.set(f"Total Value: PHP {total}")

def video_loop():
    global detected_bills, total

    cap = cv2.VideoCapture(1)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame = imutils.resize(frame, width=640)
        output_frame, detected_bills, total = detect_bills_by_number_color(frame)

        update_gui()

        cv2.putText(output_frame, f"Total Bills: {len(detected_bills)}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
        cv2.putText(output_frame, f"Total Value: PHP {total}", (10, 70),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)

        cv2.imshow("Bill Detection", output_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    root.quit()  

root = tk.Tk()
root.title("Real-Time PHP Bill Detection")
root.geometry("300x150")
root.resizable(False, False)

total_bills_var = tk.StringVar()
total_value_var = tk.StringVar()

tk.Label(root, text="Detected Bills:", font=("Arial", 14)).pack(pady=10)
tk.Label(root, textvariable=total_bills_var, font=("Arial", 12)).pack()
tk.Label(root, textvariable=total_value_var, font=("Arial", 12)).pack()


detected_bills = []
total = 0

video_thread = Thread(target=video_loop)
video_thread.daemon = True
video_thread.start()

root.mainloop()


: 